In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

In [167]:
#Declare filepath of dataset
filepath = '/Users/christopheralpert/Documents/Eductation/CompFinance/CFRM523/rep_project/data/US Wheat Futures Historical Data.csv'

In [168]:
#Load Data
df = load_data(filepath)

#First change 'Price' to 'Close'
df.rename(columns={'Price': 'Close'}, inplace=True)

#Reorder data so oldest is at top, newest is at bottom
df = df.iloc[::-1].reset_index(drop=True)

#Clean Data
df.fillna(method='ffill', inplace=True)

In [169]:
#Inspect
df

,Date,Close,Open,High,Low,Vol.,Change %
0,02/01/1995,370.50,373.50,376.00,370.25,7.51K,-0.80%
1,02/02/1995,376.75,369.00,377.00,367.50,11.62K,1.69%
2,02/03/1995,372.25,376.50,377.25,370.50,9.50K,-1.19%
3,02/06/1995,369.75,372.50,374.75,369.00,4.57K,-0.67%
4,02/07/1995,373.25,370.00,374.00,369.75,5.50K,0.95%
...,...,...,...,...,...,...,...
4995,02/13/2014,594.50,586.38,596.63,585.38,38.27K,1.47%
4996,02/14/2014,597.50,594.88,602.88,594.50,38.27K,0.50%
4997,02/18/2014,612.13,599.25,613.13,598.88,38.27K,2.45%
4998,02/19/2014,620.38,611.88,620.38,609.88,38.27K,1.35%


In [171]:
#Before feature engineering, let's convert all types to float or int. Create separate columns for 'Date' and drop 'Date'

# Convert 'Close' column to numeric, coercing errors
df['Close'] = pd.to_numeric(df['Close'], errors='coerce')

# You might have NaNs where there were non-numeric strings, fill them if necessary
df['Close'].fillna(method='ffill', inplace=True)  # Forward fill to handle NaNs, or choose another method

# Convert 'Date' column to datetime format
#df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Now you can extract datetime components
#df['Year'] = df['Date'].dt.year
#df['Month'] = df['Date'].dt.month
#df['Day'] = df['Date'].dt.day
#df['day_of_week'] = df['Date'].dt.dayofweek
# Now drop the original 'Date' column
#df.drop('Date', axis=1, inplace=True)


# Explicitly convert to string, replace commas, and convert to float
def clean_numeric(column):
    # Convert to string first to ensure .str operations can be performed
    column = column.astype(str)
    # Replace commas and convert to float
    column = column.str.replace(',', '')
    return pd.to_numeric(column, errors='coerce')

# Apply this function to the columns known to contain numeric strings with commas
columns_to_clean = ['Open','High','Low']  # Extend this list based on your actual data
for col in columns_to_clean:
    df[col] = clean_numeric(df[col])


#Convert
def convert_volume(vol):
    # Ensure vol is a string before checking for 'K' or 'M'
    vol = str(vol)
    if vol == '-' or vol == 'nan':  # Handle both '-' and unexpected NaN values represented as strings
        return np.nan
    elif 'K' in vol:
        return float(vol.replace('K', '')) * 1e3
    elif 'M' in vol:
        return float(vol.replace('M', '')) * 1e6
    else:
        try:
            return float(vol)  # Handle numbers as strings or normal numbers
        except ValueError:
            return np.nan  # In case of any other unexpected string that cannot be converted

# Apply this function to the 'Vol.' column
df['Vol.'] = df['Vol.'].apply(convert_volume)

# Option to fill NaN values, choose one:
# df['Vol.'] = df['Vol.'].fillna(0)  # Replace NaN with 0
df['Vol.'] = df['Vol.'].fillna(df['Vol.'].mean())  # Replace NaN with mean


#Drop the % sign from 'change %' and convert to float

# Convert entire 'Change %' column to string, handling potential NaN values
df['Change %'] = df['Change %'].astype(str)

# Strip the '%' character and convert to float, dividing by 100 to turn into a decimal
df['Change %'] = df['Change %'].str.rstrip('%').astype('float') / 100.0

# Check for entries that were originally NaN and set them back to NaN if they were converted to strings
df['Change %'] = df['Change %'].replace(-0.01, np.nan)  # Assuming '-0.01' results from empty entries turned into '-%'

print(df.dtypes)

Date         object
Close       float64
Open        float64
High        float64
Low         float64
Vol.        float64
Change %    float64
dtype: object


In [172]:
#### FEATURE ENGINEERING ######

#Create a returns feature for three different periods

#Run a simple for loop which calculates returns
#Define 3 periods of interest: daily, monthly and quarterly

periods = [1, 21, 63]
for period in periods:
        df[f'return_{period}d'] = df['Close'].pct_change(periods=period).fillna(0)
        

#Create a MACD feature

span1=12
span2=26
signal=9

exp1 = df['Close'].ewm(span=span1, adjust=False).mean()
exp2 = df['Close'].ewm(span=span2, adjust=False).mean()
df['macd'] = exp1 - exp2
df['macd_signal'] = df['macd'].ewm(span=signal, adjust=False).mean()
df['macd_hist'] = df['macd'] - df['macd_signal']

#Create a volatility feature
window=10
df['volatility'] = df['return_1d'].rolling(window=window).std()

In [184]:
#Create a Target column

# Shift the 'Price' column by -1 to compare the next day's closing price to the current day
df['Target'] = (df['Close'].shift(-1) > df['Close']).astype(int)


In [185]:
#Inspect and verify target is accurate
df

,Date,Close,Open,High,Low,Vol.,Change %,return_1d,return_21d,return_63d,macd,macd_signal,macd_hist,volatility,Target
0,02/01/1995,-0.507140,-0.486610,-0.497744,-0.479235,-0.387881,-0.371484,-0.015586,-0.064437,-0.113838,-0.026005,-0.027676,-0.000001,NaN,1
1,02/02/1995,-0.474837,-0.509443,-0.492779,-0.493480,-0.298038,0.734361,0.751372,-0.064437,-0.113838,0.017229,-0.018474,0.112992,NaN,0
2,02/03/1995,-0.498095,-0.471389,-0.491538,-0.477940,-0.344380,-0.544688,-0.558635,-0.064437,-0.113838,0.019481,-0.010632,0.096278,NaN,0
3,02/06/1995,-0.511016,-0.491684,-0.503950,-0.485710,-0.452148,-0.313749,-0.320927,-0.064437,-0.113838,0.003433,-0.007775,0.035081,NaN,1
4,02/07/1995,-0.492927,-0.504369,-0.507674,-0.481825,-0.431818,0.405716,0.414782,-0.064437,-0.113838,0.014736,-0.003083,0.057604,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,02/13/2014,0.650587,0.593536,0.597665,0.635127,0.284517,0.636656,0.653340,0.234105,-0.650730,-0.113907,-0.588542,1.434463,-0.499596,1
4996,02/14/2014,0.666093,0.636665,0.628696,0.682368,0.284517,0.205865,0.213843,0.542856,-0.607582,0.008087,-0.469112,1.466400,-0.507162,1
4997,02/18/2014,0.741707,0.658838,0.679586,0.705056,0.284517,1.071888,1.097647,0.735365,-0.437462,0.204482,-0.331765,1.686395,-0.432448,1
4998,02/19/2014,0.784346,0.722922,0.715582,0.762036,0.284517,0.583362,0.597173,1.104172,-0.427007,0.412795,-0.177547,1.893541,-0.801462,0


In [187]:
#Normalize the features
scaler = StandardScaler()
#if 'Date' in df.columns:
    #df['Date'] = pd.to_datetime(df['Date'])
    #df['Month'] = df['Date'].dt.month
    #df['Day'] = df['Date'].dt.day
    #df['Weekday'] = df['Date'].dt.weekday
    #df.drop('Date', axis=1, inplace=True)  # Optional: remove if not needed for modeling

#Select only numerical columns and exclude the target column for scaling
features = df.columns[(df.dtypes == 'float64') | (df.dtypes == 'int64')]
features = (features[features != 'Target'])# & (features[features != 'Date'])   # Exclude the target column from scaling

df[features] = scaler.fit_transform(df[features])

In [189]:
#Inspect
df

,Date,Close,Open,High,Low,Vol.,Change %,return_1d,return_21d,return_63d,macd,macd_signal,macd_hist,volatility,Target
0,02/01/1995,-0.507140,-0.486610,-0.497744,-0.479235,-0.387881,-0.371484,-0.015586,-0.064437,-0.113838,-0.026005,-0.027676,-0.000001,NaN,1
1,02/02/1995,-0.474837,-0.509443,-0.492779,-0.493480,-0.298038,0.734361,0.751372,-0.064437,-0.113838,0.017229,-0.018474,0.112992,NaN,0
2,02/03/1995,-0.498095,-0.471389,-0.491538,-0.477940,-0.344380,-0.544688,-0.558635,-0.064437,-0.113838,0.019481,-0.010632,0.096278,NaN,0
3,02/06/1995,-0.511016,-0.491684,-0.503950,-0.485710,-0.452148,-0.313749,-0.320927,-0.064437,-0.113838,0.003433,-0.007775,0.035081,NaN,1
4,02/07/1995,-0.492927,-0.504369,-0.507674,-0.481825,-0.431818,0.405716,0.414782,-0.064437,-0.113838,0.014736,-0.003083,0.057604,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,02/13/2014,0.650587,0.593536,0.597665,0.635127,0.284517,0.636656,0.653340,0.234105,-0.650730,-0.113907,-0.588542,1.434463,-0.499596,1
4996,02/14/2014,0.666093,0.636665,0.628696,0.682368,0.284517,0.205865,0.213843,0.542856,-0.607582,0.008087,-0.469112,1.466400,-0.507162,1
4997,02/18/2014,0.741707,0.658838,0.679586,0.705056,0.284517,1.071888,1.097647,0.735365,-0.437462,0.204482,-0.331765,1.686395,-0.432448,1
4998,02/19/2014,0.784346,0.722922,0.715582,0.762036,0.284517,0.583362,0.597173,1.104172,-0.427007,0.412795,-0.177547,1.893541,-0.801462,0


In [192]:
#Drop the last row where future price is unknown due to shift
clean_df = clean_df.iloc[:-1]

In [193]:
clean_df

,Date,Close,Open,High,Low,Vol.,Change %,return_1d,return_21d,return_63d,macd,macd_signal,macd_hist,volatility,Target
9,02/14/1995,-0.503264,-0.495490,-0.507674,-0.483120,-0.449743,-0.073927,-0.076736,-0.064437,-0.113838,0.007077,0.001755,0.017732,-0.730385,0
10,02/15/1995,-0.529106,-0.500564,-0.522569,-0.506429,-0.324925,-0.615747,-0.627913,-0.064437,-0.113838,-0.032836,-0.005585,-0.090127,-0.657557,1
11,02/16/1995,-0.526522,-0.522128,-0.538705,-0.514199,-0.406680,0.045984,0.046483,-0.064437,-0.113838,-0.060570,-0.017360,-0.144584,-0.808153,0
12,02/17/1995,-0.526522,-0.519591,-0.537464,-0.506429,-0.447994,-0.016192,-0.015586,-0.064437,-0.113838,-0.081905,-0.031322,-0.171426,-0.853840,0
13,02/21/1995,-0.558824,-0.524665,-0.547393,-0.532329,-0.374328,-0.771186,-0.790386,-0.064437,-0.113838,-0.141218,-0.055116,-0.292156,-0.756758,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4994,02/12/2014,0.606036,0.608099,0.604467,0.637717,0.284517,-0.260455,-0.266400,0.195159,-0.746172,-0.241849,-0.705371,1.375110,-0.522140,1
4995,02/13/2014,0.650587,0.593536,0.597665,0.635127,0.284517,0.636656,0.653340,0.234105,-0.650730,-0.113907,-0.588542,1.434463,-0.499596,1
4996,02/14/2014,0.666093,0.636665,0.628696,0.682368,0.284517,0.205865,0.213843,0.542856,-0.607582,0.008087,-0.469112,1.466400,-0.507162,1
4997,02/18/2014,0.741707,0.658838,0.679586,0.705056,0.284517,1.071888,1.097647,0.735365,-0.437462,0.204482,-0.331765,1.686395,-0.432448,1


In [194]:
#Export the df....if running Train Test Split in your model script
clean_df.to_csv('TrainingSets/TrainingData.csv', index=False)